# MIT iQuHack Microsoft Challenge: Optimizing Quantum Oracles, Task 3

To work on this task,
1. Use the notebook for this task. Each of the notebooks in the repository has the code of the corresponding task.
2. Update your team name and Slack ID variables in the next code cell (you can use different Slack IDs for different tasks if different team members work on them, but make sure to use the same team name throughout the Hackathon). Do not change the task variable!
3. Work on your task in the cell that contains operation `Task3`! Your goal is to rewrite the code so that it maintains its correctness, but requires as few resources as possible. See `evaluate_results` function for details on how your absolute score for the task is calculated.
4. Submit your task using qBraid. Use the Share Notebook feature on qBraid (See File > Share Notebook) and enter the email rickyyoung@qbraid.com.   Once you click submit, if the share notebook feature works correctly, it should show that you receive no errors and the email you entered will disappear. 

Log in to Azure (once per session, don't need to do it if running from Azure workspace)

In [1]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RHGXSD2GV to authenticate.
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
3cbe47e5-84c4-481c-9a4e-f136937400d7 'Default Directory'
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "93e46887-8cc6-43a4-a4e0-e2209181e7d0",
    "id": "464e0e7f-8636-45a7-9148-119678af07d3",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "d0ecd01b-d782-448e-bae0-c3cad0e0543a"
      },
      {
        "tenantId": "94c4857e-1130-4ab8-8eac-069b40c9db20"
      },
      {
        "tenantId": "f702a9dc-ae48-4dc7-8f0a-8155a6dfa4e5"
      }
    ],
    "name": "MIT iQuHack backup_Kaliappan_Yadav",
    "state": "Enabled",
    "tenantId": "93e46887-8cc6-43a4-a4e0-e2209181e7d0",
    "user": {
      "name": "skaliappan1908@gmail.com",
      "type": "user"
    }
  }
]


## Step 1. Write the code

In [2]:
# Run this code cell to import the modules required to work with Q# and Azure
import qsharp
from qsharp import azure

Preparing Q# environment...


In [3]:
teamname="QuantumNerds"  # Update this field with your team name
task=["task3"]
slack_id="U04JVBHPENS"         # Update this field with Slack ID of the person who worked on this task as the troubleshooting contact

In [4]:
# You don't need to run this cell, it defines Q# operations as Python types to keep IntelliSense happy
Task3_DumpMachineWrapper : qsharp.QSharpCallable = None
Task3_ResourceEstimationWrapper : qsharp.QSharpCallable = None

**The complete code for Task 3 should be in this cell.**   
This cell can include additional open statements and helper operations and functions if your solution needs them.  
If you define helper operations in other cells, they will not be picked up by the grader!

In [17]:
%%qsharp
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;

operation Task3(input : Qubit[], target : Qubit) : Unit is Adj {
  within {
        X(input[5]);
        X(input[4]);
    }
    apply {
        Controlled Task3_msb_00([input[5], input[4]], (input[...3], target));
    }
    Controlled Task3_msb_11([input[5], input[4]], (input[...3], target)); 
    
    within {
        CNOT(input[5], input[4]); 
    }
    apply {
        Controlled Task3_01([input[4]], (input[...3], target)); // top bits: 01 or 10
    }
}

operation Task3_0(input : Qubit[], target : Qubit) : Unit is Adj + Ctl {
    
    X(input[4]);
    Controlled Task3_msb_00([input[4]], (input[...3], target));
    X(input[4]);
    Controlled Task3_01([input[4]], (input[...3], target));
}

operation Task3_msb_00(input : Qubit[], target : Qubit) : Unit is Adj + Ctl {
    for i in [7, 11, 13, 14] {
        ControlledOnInt(i, X)(input, target);
    }
}

operation Task3_01(input : Qubit[], target : Qubit) : Unit is Adj + Ctl {
    for i in [3, 5, 6, 9, 10, 12] {
        ControlledOnInt(i, X)(input, target);
    }
}

operation Task3_1(input : Qubit[], target : Qubit) : Unit is Adj + Ctl {
    X(input[4]);
    Controlled Task3_01([input[4]], (input[...3], target));
    X(input[4]);
    Controlled Task3_msb_11([input[4]], (input[...3], target));
}

operation Task3_msb_11(input : Qubit[], target : Qubit) : Unit is Adj + Ctl {
    for i in [1, 2, 4, 8] {
        ControlledOnInt(i, X)(input, target);
    }
}

In [18]:
%%qsharp
// Wrapper operation that allows you to observe the effects of the marking oracle by running it on a simulator.
operation Task3_DumpMachineWrapper() : Unit {
    let N = 6;
    use (input, target) = (Qubit[N], Qubit());
    // Prepare an equal superposition of all input states in the input register.
    ApplyToEach(H, input);
    // Apply the oracle.
    Task3(input, target);
    // Print the state of the system after the oracle application.
    DumpMachine();
    ResetAll(input + [target]);
}

// Wrapper operation that allows to run resource estimation for the task.
// This operation only allocates the qubits and applies the oracle once, not using any additional gates or measurements.
operation Task3_ResourceEstimationWrapper() : Unit {
    let N = 6;
    use (input, target) = (Qubit[N], Qubit());
    Task3(input, target);
}

## Step 2. Run the code on a simulator to see what it does
You can also write your own code to explore the effects of the oracle (for example, applying it to different basis states and measuring the results).

In [19]:
# Note that in the output of this cell the target qubit corresponds to the rightmost bit
qsharp.config["dump.basisStateLabelingConvention"]="Bitstring"
qsharp.config["dump.phaseDisplayStyle"]="None"
# Uncomment the following line if you want to see only the entries with non-zero amplitudes
# qsharp.config["dump.truncateSmallAmplitudes"]=True
Task3_DumpMachineWrapper.simulate()

|0000000⟩	0.12500000000000008 + 0𝑖
|0000001⟩	0 + 0𝑖
|0000010⟩	0.12500000000000008 + 0𝑖
|0000011⟩	0 + 0𝑖
|0000100⟩	0.12500000000000008 + 0𝑖
|0000101⟩	0 + 0𝑖
|0000110⟩	0.12500000000000008 + 0𝑖
|0000111⟩	0 + 0𝑖
|0001000⟩	0.12500000000000008 + 0𝑖
|0001001⟩	0 + 0𝑖
|0001010⟩	0.12500000000000008 + 0𝑖
|0001011⟩	0 + 0𝑖
|0001100⟩	0.12500000000000008 + 0𝑖
|0001101⟩	0 + 0𝑖
|0001110⟩	0 + 0𝑖
|0001111⟩	0.12500000000000008 + 0𝑖
|0010000⟩	0.12500000000000008 + 0𝑖
|0010001⟩	0 + 0𝑖
|0010010⟩	0.12500000000000008 + 0𝑖
|0010011⟩	0 + 0𝑖
|0010100⟩	0.12500000000000008 + 0𝑖
|0010101⟩	0 + 0𝑖
|0010110⟩	0 + 0𝑖
|0010111⟩	0.12500000000000008 + 0𝑖
|0011000⟩	0.12500000000000008 + 0𝑖
|0011001⟩	0 + 0𝑖
|0011010⟩	0 + 0𝑖
|0011011⟩	0.12500000000000008 + 0𝑖
|0011100⟩	0 + 0𝑖
|0011101⟩	0.12500000000000008 + 0𝑖
|0011110⟩	0.12500000000000008 + 0𝑖
|0011111⟩	0 + 0𝑖
|0100000⟩	0.12500000000000008 + 0𝑖
|0100001⟩	0 + 0𝑖
|0100010⟩	0.12500000000000008 + 0𝑖
|0100011⟩	0 + 0𝑖
|0100100⟩	0.12500000000000008 + 0𝑖
|0100101⟩	0 + 0𝑖
|0100110⟩	0 + 0𝑖
|0100111⟩	0.12500000000000008 + 0𝑖
|0101000⟩	0.12500000000000008 + 0𝑖
|0101001⟩	0 + 0𝑖
|0101010⟩	0 + 0𝑖
|0101011⟩	0.12500000000000008 + 0𝑖
|0101100⟩	0 + 0𝑖
|0101101⟩	0.12500000000000008 + 0𝑖
|0101110⟩	0.12500000000000008 + 0𝑖
|0101111⟩	0 + 0𝑖
|0110000⟩	0.12500000000000008 + 0𝑖
|0110001⟩	0 + 0𝑖
|0110010⟩	0 + 0𝑖
|0110011⟩	0.12500000000000008 + 0𝑖
|0110100⟩	0 + 0𝑖
|0110101⟩	0.12500000000000008 + 0𝑖
|0110110⟩	0.12500000000000008 + 0𝑖
|0110111⟩	0 + 0𝑖
|0111000⟩	0 + 0𝑖
|0111001⟩	0.12500000000000008 + 0𝑖
|0111010⟩	0.12500000000000008 + 0𝑖
|0111011⟩	0 + 0𝑖
|0111100⟩	0.12500000000000008 + 0𝑖
|0111101⟩	0 + 0𝑖
|0111110⟩	0.12500000000000008 + 0𝑖
|0111111⟩	0 + 0𝑖
|1000000⟩	0.12500000000000008 + 0𝑖
|1000001⟩	0 + 0𝑖
|1000010⟩	0.12500000000000008 + 0𝑖
|1000011⟩	0 + 0𝑖
|1000100⟩	0.12500000000000008 + 0𝑖
|1000101⟩	0 + 0𝑖
|1000110⟩	0 + 0𝑖
|1000111⟩	0.12500000000000008 + 0𝑖
|1001000⟩	0.12500000000000008 + 0𝑖
|1001001⟩	0 + 0𝑖
|1001010⟩	0 + 0𝑖
|1001011⟩	0.12500000000000008 + 0𝑖
|1001100⟩	0 + 0𝑖
|1001101⟩	0.12500000000000008 + 0𝑖
|1001110⟩	0.12500000000000008 + 0𝑖
|1001111⟩	0 + 0𝑖
|1010000⟩	0.12500000000000008 + 0𝑖
|1010001⟩	0 + 0𝑖
|1010010⟩	0 + 0𝑖
|1010011⟩	0.12500000000000008 + 0𝑖
|1010100⟩	0 + 0𝑖
|1010101⟩	0.12500000000000008 + 0𝑖
|1010110⟩	0.12500000000000008 + 0𝑖
|1010111⟩	0 + 0𝑖
|1011000⟩	0 + 0𝑖
|1011001⟩	0.12500000000000008 + 0𝑖
|1011010⟩	0.12500000000000008 + 0𝑖
|1011011⟩	0 + 0𝑖
|1011100⟩	0.12500000000000008 + 0𝑖
|1011101⟩	0 + 0𝑖
|1011110⟩	0.12500000000000008 + 0𝑖
|1011111⟩	0 + 0𝑖
|1100000⟩	0.12500000000000008 + 0𝑖
|1100001⟩	0 + 0𝑖
|1100010⟩	0 + 0𝑖
|1100011⟩	0.12500000000000008 + 0𝑖
|1100100⟩	0 + 0𝑖
|1100101⟩	0.12500000000000008 + 0𝑖
|1100110⟩	0.12500000000000008 + 0𝑖
|1100111⟩	0 + 0𝑖
|1101000⟩	0 + 0𝑖
|1101001⟩	0.12500000000000008 + 0𝑖
|1101010⟩	0.12500000000000008 + 0𝑖
|1101011⟩	0 + 0𝑖
|1101100⟩	0.12500000000000008 + 0𝑖
|1101101⟩	0 + 0𝑖
|1101110⟩	0.12500000000000008 + 0𝑖
|1101111⟩	0 + 0𝑖
|1110000⟩	0 + 0𝑖
|1110001⟩	0.12500000000000008 + 0𝑖
|1110010⟩	0.12500000000000008 + 0𝑖
|1110011⟩	0 + 0𝑖
|1110100⟩	0.12500000000000008 + 0𝑖
|1110101⟩	0 + 0𝑖
|1110110⟩	0.12500000000000008 + 0𝑖
|1110111⟩	0 + 0𝑖
|1111000⟩	0.12500000000000008 + 0𝑖
|1111001⟩	0 + 0𝑖
|1111010⟩	0.12500000000000008 + 0𝑖
|1111011⟩	0 + 0𝑖
|1111100⟩	0.12500000000000008 + 0𝑖
|1111101⟩	0 + 0𝑖
|1111110⟩	0.12500000000000008 + 0𝑖
|1111111⟩	0 + 0𝑖

()

## Step 3. Evaluate the code using resource estimation

In [9]:
qsharp.azure.connect(
    resourceId="/subscriptions/464e0e7f-8636-45a7-9148-119678af07d3/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/QuantumNerds",
    location="East US",
    credential="CLI")

Connecting to Azure Quantum...

Connected to Azure Quantum workspace QuantumNerds in location eastus.


[{'id': 'ionq.qpu', 'current_availability': {}, 'average_queue_time': 136163},
 {'id': 'ionq.qpu.aria-1', 'current_availability': {}, 'average_queue_time': 448072},
 {'id': 'ionq.simulator', 'current_availability': {}, 'average_queue_time': 2},
 {'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s1-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s2', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.hqs-lt-s2-apival', 'current_availability': {}, 'average_queue_time': 1},
 {'id': 'quantinuum.hqs-lt-s1-sim', 'current_availability': {}, 'average_queue_time': 70},
 {'id': 'quantinuum.hqs-lt-s2-sim', 'current_availability': {}, 'average_queue_time': 177},
 {'id': 'quantinuum.hqs-lt', 'current_availability': {}, 'average_queue_time': 0},
 {'id': 'quantinuum.qpu.h1-1', 'current_availa

In [10]:
qsharp.azure.target("microsoft.estimator")

Loading package Microsoft.Quantum.Providers.Core and dependencies...
Active target is now microsoft.estimator


{'id': 'microsoft.estimator', 'current_availability': {}, 'average_queue_time': 0}

In [20]:
# Update job name to a more descriptive string to make it easier to find it in Job Management tab later
result = qsharp.azure.execute(Task3_ResourceEstimationWrapper, jobName="RE for the task 3")

Submitting Task3_ResourceEstimationWrapper to target microsoft.estimator...
Job successfully submitted.
   Job name: RE for the task 3
   Job ID: aeaf2230-1729-45ee-abf3-8ce8dcd056ef
Waiting up to 30 seconds for Azure Quantum job to complete...
[9:32:18 AM] Current job status: Executing
[9:32:23 AM] Current job status: Succeeded


In [21]:
# If you need to pull up the results of an old job, use its job ID with qsharp.azure.output command
# result = qsharp.azure.output("...")
result

Physical qubits,143506,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 7986 physical qubits to implement the algorithm logic, and 135520 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Runtime,1ms 64us 800ns,"Total runtime This is a runtime estimate (in nanosecond precision) for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (4us 400ns) multiplied by the 242 logical cycles to run the algorithm. If however the duration of a single T factory (here: 57us 200ns) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
Logical algorithmic qubits,33,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the \(Q_{\rm alg} = 11\) logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 33$ logical qubits."
Algorithmic depth,242,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 50 single-qubit measurements, the 0 arbitrary single-qubit rotations, and the 0 T gates, three multi-qubit measurements for each of the 14 CCZ and 50 CCiX gates in the input program, as well as No rotations in algorithm multi-qubit measurements for each of the 0 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,242,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 242. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Number of T states,256,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 0 T gates, four T states for each of the 14 CCZ and 50 CCiX gates, as well as No rotations in algorithm for each of the 0 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,14,Number of T factories capable of producing the demanded 256 T states during the algorithm's runtime The total number of T factories 14 that are executed in parallel is computed as \(\left\lceil\dfrac{256\;\text{T states} \cdot 57us 200ns\;\text{T factory duration}}{1\;\text{T states per T factory} \cdot 1ms 64us 800ns\;\text{algorithm runtime}}\right\rceil\)
Number of T factory invocations,19,"Number of times all T factories are invoked In order to prepare the 256 T states, the 14 copies of the T factory are repeatedly invoked 19 times."
Physical algorithmic qubits,7986,Number of physical qubits for the algorithm after layout The 7986 are the product of the 33 logical qubits after layout and the 242 physical qubits that encode a single logical qubit.
Physical T factory qubits,135520,"Number of physical qubits for the T factories Each T factory requires 9680 physical qubits and we run 14 in parallel, therefore we need $135520 = 9680 \cdot 14$ qubits."
Required logical qubit error rate,6.26e-8,The minimum logical qubit error rate required to run the algorithm within the error budget The minimum logical qubit error rate is obtained by dividing the logical error probability 5.00e-4 by the product of 33 logical qubits and the total cycle count 242.


In [13]:
# The function that extracts the relevant resource information from the resource estimation job results and produces your absolute score.
def evaluate_results(res) : 
    width = res['physicalCounts']['breakdown']['algorithmicLogicalQubits']
    depth = res['physicalCounts']['breakdown']['algorithmicLogicalDepth']
    print(f"Logical algorithmic qubits = {width}")
    print(f"Algorithmic depth = {depth}")
    print(f"Score = {width * depth}")
    return width * depth


In [22]:
evaluate_results(result)

Logical algorithmic qubits = 33
Algorithmic depth = 242
Score = 7986


7986